# Part 4: TPOT

Without an extensive background in the statistics and mathematics behind different machine learning models, it can be difficult to determine what the best model for a given dataset is. This also applies to tuning the parameters. As you have probably noticed, the models we've used in this workshop so far have many different parameters, and it's by no means obvious how to tune them. 

Moreover, testing out many different models, along with many different combinations of parameters, could be extremely time consuming and impractical. 

[TPOT](https://github.com/rhiever/tpot) is a new tool that automates the model selection and hyperparameter tuning process using genetic programming. It also determines what preprocessing, if any, is necessary, such as PCA or standard scaling. It then exports this model to a file with the scikit-learn code written for you. 

Although it is in your best interest to learn as much about the theory behind machine learning as possible, tools like TPOT can theoretically do the work for you. 

TPOT can be used for both classification and regression.

Let's set the random seed.

In [ ]:
import numpy as np

np.random.seed(10)

## Classification

In [ ]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target,
                                                    train_size=0.75, test_size=0.25)

In [ ]:
!pip install tpot
from tpot import TPOTClassifier

tpot = TPOTClassifier(generations=5, population_size=20, verbosity=1)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_cancer_pipeline.py')

     |████████████████████████████████| 92kB 4.2MB/s 
     |████████████████████████████████| 163kB 17.0MB/s 
     |████████████████████████████████| 166.7MB 28kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp37-none-any.whl size=11954 sha256=239941e359327494b06cd0d5b12246f97579302f4d69ea929db790d4bb5ec718
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit
  Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Best pipeline: ExtraTreesClassifier(PolynomialFeatures(input_matrix, degree=2, include_bias=False, interaction_only=False), bootstrap=False, criterion=gini, max_features=0.35000000000000003, min_samples_leaf=3, min_samples_split=8, n_estimators=100)
0.986013986013986


Let's look at the model TPOT created for us:

In [ ]:
!cat tpot_cancer_pipeline.py

import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=None)

# Average CV score on the training set was: 0.9670861833105336
exported_pipeline = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False, interaction_only=False),
    ExtraTreesClassifier(bootstrap=False, criterion="gini", max_features=0.35000000000000003, min_samples_leaf=3, min_samples_split=8, n_estimators=100)
)

exported_pipeline.fit(training_features, training_target)
results 

## Regression

First we'll load the preprocessed heart data that we created during the regression tutorial.

In [ ]:
heart_data = np.load('data/heart_preproc.npz')

X_train = heart_data['X_train']
X_test = heart_data['X_test']
y_train = heart_data['y_train']
y_test = heart_data['y_test']

Now TPOT will make the model.

In [ ]:
from tpot import TPOTRegressor

tpot = TPOTRegressor(generations=5, population_size=20, verbosity=1, scoring='r2')  # generations for optimization, pop size is models
tpot.fit(X_train, y_train.ravel())
print(tpot.score(X_test, y_test.ravel()))
tpot.export('tpot_heart_pipeline.py')

In [ ]:
!cat tpot_heart_pipeline.py